# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [25]:
import pandas as pd
import numpy as np
from scipy import stats

In [26]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [27]:
# number of callbacks for names
call_black = sum(data[data.race=='b'].call)
#print(call_black)
call_white = sum(data[data.race=='w'].call)
print(call_white)
black_total = len(data[data['race']=='b'])
#print(black_total)
white_total = len(data[data['race']=='w'])
#print(white_total)

235.0


In [28]:
data.info
#1. The z-test ClT is appropriate for this, considering that there are 4870 data points.

<bound method DataFrame.info of      id   ad  education  ofjobs  yearsexp  honors  volunteer  military  \
0     b    1          4       2         6       0          0         0   
1     b    1          3       3         6       0          1         1   
2     b    1          4       1         6       0          0         0   
3     b    1          3       4         6       0          1         0   
4     b    1          3       3        22       0          0         0   
5     b    1          4       2         6       1          0         0   
6     b    1          4       2         5       0          1         0   
7     b    1          3       4        21       0          1         0   
8     b    1          4       3         3       0          0         0   
9     b    1          4       2         6       0          1         0   
10    b    1          4       4         8       0          1         0   
11    b    1          4       4         8       0          0         0   
12    

In [48]:
#2. The null hypothesis is that there is no difference between callbacks in white- and black-sounding names.
#3. Compute margin of error, confidence interval, and p-value.

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
       ... 
4840    0.0
4841    0.0
4842    0.0
4843    1.0
4844    0.0
4845    0.0
4846    1.0
4847    1.0
4848    1.0
4849    0.0
4850    0.0
4851    0.0
4852    0.0
4853    0.0
4854    0.0
4855    0.0
4856    0.0
4857    0.0
4858    0.0
4859    1.0
4860    0.0
4861    1.0
4862    0.0
4863    0.0
4864    0.0
4865    0.0
4866    0.0
4867    0.0
4868    0.0
4869    0.0
Name: call, Length: 4870, dtype: float32

In [46]:
data.columns
#Prepare bootstrapping functions

def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))


def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [53]:
#create an array for black and white, then bootstrap 10000 from them and see what the results from

##race_array = []
#for cell in data['race'] :
    #if cell == 'b' :
        #race_array.append(1)
    #else :
        #race_array.append(0)

array_black = data[data.race=='b']
array_white = data[data.race=='w']
array_black = array_black['call']
array_white = array_white['call']
#print(array_white)

black_bs_reps = draw_bs_reps(array_black, np.mean, 10000)
white_bs_reps = draw_bs_reps(array_white, np.mean, 10000)
print(black_bs_reps)

[ 0.06776181  0.05749487  0.06940452 ...,  0.062423    0.062423    0.05790554]


In [59]:

white_mean = np.mean(array_white)
black_mean = np.mean(array_black)
#So is this significant? Let's find out.
print(black_mean)


0.0644763857126236


In [69]:
#Test between white and black applicants. 2-sample bootstrapped t-test

overall_mean = np.mean(data['call'])

real_diff_means = np.mean(array_black) - np.mean(array_white)
#Generate shifted arrays and execute

black_shifted = array_black - np.mean(array_black) + overall_mean
white_shifted = array_white - np.mean(array_white) + overall_mean

black_replicates = draw_bs_reps(black_shifted, np.mean, 10000)
white_replicates = draw_bs_reps(white_shifted, np.mean, 10000)




bs_replicates = black_replicates - white_replicates
print(bs_replicates)

p = np.sum(bs_replicates < real_diff_means) / 10000
print(p)

#The p value is low, very very low, which suggests that there should be a significant difference between
#race and callbacks for job interviews. Out of 10,000 bootstrapped trials, none of them experienced a 
#difference as great as the real difference when their means were shifted - this kind of result is not
#left up to chance.

[ 0.00492816 -0.00123201  0.00821358 ...,  0.00164273 -0.00164268
  0.00616018]
0.0


In [ ]:
#5. There is, however, no guarantee that race is the *most* important factor when considering job 
#application success. In order to test such a hypothesis, many more comparison tests would have to be
#performed with different variables in mind in order to consider effect size. p-test does not provide
#a scale of the influence of a variable, just whether the differences in variables is significant.
#I would set up a linear regression test with many variables that I think may be signficant in order to
#truly test that.